First CyClip Attempt, very poor results

In [ ]:
def forward(self, batch):
    # Getting Image and Text Features
    image_features = self.image_encoder(batch["image"])
    text_features = self.text_encoder(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

    # Getting Image and Text Embeddings (with same dimension)
    image_embeddings = self.image_projection(image_features)
    text_embeddings = self.text_projection(text_features)
    criterion = nn.CrossEntropyLoss(reduction = "sum").to(CFG.device)

    # Calculating the Loss
    logits = (text_embeddings @ image_embeddings.T) / self.temperature
    targets = torch.arange(batch['input_ids'].shape[0]).to(CFG.device)
    contrastive_loss = (criterion(logits, targets) + criterion(logits.T, targets.T)) / 2

    # In-modal consistency
    logits_image_per_image = image_embeddings @ image_embeddings.T / self.temperature
    logits_text_per_text = text_embeddings @ text_embeddings.T / self.temperature
    inmodal_cyclic_loss = (logits_image_per_image - logits_text_per_text).square().mean() * (self.temperature * self.temperature) * CFG.batch_size

    # Cross-modal consistency
    logits_text_per_image = image_embeddings @ text_embeddings.T / self.temperature
    logits_image_per_text = logits_text_per_image.T
    crossmodal_cyclic_loss = (logits_text_per_image - logits_image_per_text).square().mean() * (self.temperature * self.temperature) * CFG.batch_size

    cyclic_loss = CFG.cylambda1 * inmodal_cyclic_loss + CFG.cylambda2 * crossmodal_cyclic_loss
    loss = contrastive_loss + cyclic_loss

    correct_preds = num_of_correct_preds(logits, batch['caption'])

    return loss, correct_preds

No CyClip, but reformatted the loss function to with same code as in the paper. Gave equal results as previous 100_pre_trained model!

In [ ]:
    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
        batch_size = batch['input_ids'].shape[0]

        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        criterion = nn.CrossEntropyLoss(reduction = "sum").to(CFG.device)

        # Cross-modal contrastive alignment (CLIP)
        logits_text_per_image = image_embeddings @ text_embeddings.T / self.temperature
        logits_image_per_text = logits_text_per_image.T
        target = torch.arange(batch_size).long().to(CFG.device, non_blocking = True)
        contrastive_loss = (criterion(logits_text_per_image, target) + criterion(logits_image_per_text, target)) / 2

        # In-modal consistency (CyCLIP)
        logits_image_per_image = image_embeddings @ image_embeddings.T / self.temperature
        logits_text_per_text = text_embeddings @ text_embeddings.T / self.temperature
        inmodal_cyclic_loss = (logits_image_per_image - logits_text_per_text).square().mean() * (self.temperature * self.temperature) * batch_size

        # Cross-modal consistency (CyCLIP)
        crossmodal_cyclic_loss = (logits_text_per_image - logits_image_per_text).square().mean() * (self.temperature * self.temperature) * batch_size

        cyclic_loss = CFG.cylambda1 * inmodal_cyclic_loss + CFG.cylambda2 * crossmodal_cyclic_loss
        loss = contrastive_loss #+ cyclic_loss

        correct_preds = num_of_correct_preds(logits_image_per_text, batch['caption'])

        return loss, correct_preds

Same as above, but with CyCLIP loss. hadde pretty poor result ending on 0.1559

In [ ]:
def forward(self, batch):
    # Getting Image and Text Features
    image_features = self.image_encoder(batch["image"])
    text_features = self.text_encoder(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

    # Getting Image and Text Embeddings (with same dimension)
    image_embeddings = self.image_projection(image_features)
    text_embeddings = self.text_projection(text_features)
    criterion = nn.CrossEntropyLoss(reduction = "sum").to(CFG.device)

    # Calculating the Loss
    logits = (text_embeddings @ image_embeddings.T) / self.temperature
    targets = torch.arange(batch['input_ids'].shape[0]).to(CFG.device)
    contrastive_loss = (criterion(logits, targets) + criterion(logits.T, targets.T)) / 2

    # In-modal consistency
    logits_image_per_image = image_embeddings @ image_embeddings.T / self.temperature
    logits_text_per_text = text_embeddings @ text_embeddings.T / self.temperature
    inmodal_cyclic_loss = (logits_image_per_image - logits_text_per_text).square().mean() * (self.temperature * self.temperature) * CFG.batch_size

    # Cross-modal consistency
    logits_text_per_image = image_embeddings @ text_embeddings.T / self.temperature
    logits_image_per_text = logits_text_per_image.T
    crossmodal_cyclic_loss = (logits_text_per_image - logits_image_per_text).square().mean() * (self.temperature * self.temperature) * CFG.batch_size

    cyclic_loss = CFG.cylambda1 * inmodal_cyclic_loss + CFG.cylambda2 * crossmodal_cyclic_loss
    loss = contrastive_loss + cyclic_loss

    correct_preds = num_of_correct_preds(logits, batch['caption'])

    return loss, correct_preds

Tried to switch loss function after 7 epochs, did not work

In [ ]:
    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
        batch_size = batch['input_ids'].shape[0]

        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        criterion = nn.CrossEntropyLoss(reduction = "sum").to(CFG.device)

        # Cross-modal contrastive alignment (CLIP)
        logits_text_per_image = image_embeddings @ text_embeddings.T / self.temperature
        logits_image_per_text = logits_text_per_image.T
        target = torch.arange(batch_size).long().to(CFG.device, non_blocking = True)
        contrastive_loss = (criterion(logits_text_per_image, target) + criterion(logits_image_per_text, target)) / 2

        # In-modal consistency (CyCLIP)
        logits_image_per_image = image_embeddings @ image_embeddings.T / self.temperature
        logits_text_per_text = text_embeddings @ text_embeddings.T / self.temperature
        inmodal_cyclic_loss = (logits_image_per_image - logits_text_per_text).square().mean() * (self.temperature * self.temperature) * batch_size

        # Cross-modal consistency (CyCLIP)
        crossmodal_cyclic_loss = (logits_text_per_image - logits_image_per_text).square().mean() * (self.temperature * self.temperature) * batch_size

        cyclic_loss = CFG.cylambda1 * inmodal_cyclic_loss + CFG.cylambda2 * crossmodal_cyclic_loss

        loss = contrastive_loss + cyclic_loss if CFG.global_epoch > 7 else contrastive_loss

        correct_preds = num_of_correct_preds(logits_image_per_text, batch['caption'])

        return loss, correct_preds